# GDP and PCE Forecasting

# Import libraries

In [4]:
from dfply import *
from sklearn import linear_model
import itertools
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace import sarimax
import warnings
warnings.filterwarnings('ignore')

# Multi Regression Model - GDP fcst

In [5]:
# Leading economic indicator
UER = pd.read_excel('IndicatorData.xlsx',sheet_name = 'QuaterlyIndicator',)
UER.index = UER.Quarter.values
UER.columns= ['Quarter','UnemploymentRate','Export','GDP','BCI']

In [1]:
gdp_fcst_loop_list_qtr = []
gdp_fcst_loop_list_semi = []
gdp_fcst_loop_list_annual = []

## Quarterly:
for i in range(91,103,1):
    
    UER_Train = UER.iloc[0:i,]
    UER_Predict = UER.iloc[i+1,]
    GDP_Train = UER_Train.GDP
    GDP_Predict = UER_Predict.GDP
    regr = linear_model.LinearRegression()
    train_x = np.asanyarray(UER_Train[['UnemploymentRate','Export','BCI']])
    train_y = np.asanyarray(GDP_Train)
    regr.fit (train_x, train_y)
    GDP_Forecast = regr.coef_[0] * UER_Predict['UnemploymentRate'] +regr.coef_[1]*UER_Predict['Export'] +regr.coef_[2]* UER_Predict['BCI'] + regr.intercept_
    
    gdp_fcst_loop_list_qtr.append(GDP_Forecast)
   
## Semi:    
for i in range(93,103,2):
    
    UER_Train = UER.iloc[0:i,]
    UER_Predict = UER.iloc[i+1,]
    GDP_Train = UER_Train.GDP
    GDP_Predict = UER_Predict.GDP
    regr = linear_model.LinearRegression()
    train_x = np.asanyarray(UER_Train[['UnemploymentRate','Export','BCI']])
    train_y = np.asanyarray(GDP_Train)
    regr.fit (train_x, train_y)
    GDP_Forecast = regr.coef_[0] * UER_Predict['UnemploymentRate'] +regr.coef_[1]*UER_Predict['Export'] +regr.coef_[2]* UER_Predict['BCI'] + regr.intercept_
    
    gdp_fcst_loop_list_semi.append(GDP_Forecast)
    
## Annual:    
for i in range(98,103,4):
    
    UER_Train = UER.iloc[0:i,]
    UER_Predict = UER.iloc[i+1,]
    GDP_Train = UER_Train.GDP
    GDP_Predict = UER_Predict.GDP
    regr = linear_model.LinearRegression()
    train_x = np.asanyarray(UER_Train[['UnemploymentRate','Export','BCI']])
    train_y = np.asanyarray(GDP_Train)
    regr.fit (train_x, train_y)
    GDP_Forecast = regr.coef_[0] * UER_Predict['UnemploymentRate'] +regr.coef_[1]*UER_Predict['Export'] +regr.coef_[2]* UER_Predict['BCI'] + regr.intercept_
    
    gdp_fcst_loop_list_annual.append(GDP_Forecast)
    
    
    
# display(gdp_fcst_loop_list_qtr,
#         gdp_fcst_loop_list_semi,
#         gdp_fcst_loop_list_annual)

NameError: name 'UER' is not defined

In [12]:
regr = linear_model.LinearRegression()
train_x = np.asanyarray(UER_Train[['UnemploymentRate','Export','BCI']])
train_y = np.asanyarray(GDP_Train)
regr.fit (train_x, train_y)
# The coefficients
# print ('Coefficients: ', regr.coef_)
# print ('Intercept: ',regr.intercept_)
# print ('Coefficients: ', regr.coef_)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [17]:
Accuracy = (GDP_Forecast-GDP_Predict)/GDP_Predict
GDP_Forecast_data = {'Forecast_value':GDP_Forecast, 'Real_value':GDP_Predict,'Accuracy':Accuracy}
GDPdata = pd.DataFrame(GDP_Forecast_data)
GDPdata.index=UER_Predict.Quarter 

## Inflation Forecast using ARIMA

In [50]:
df = pd.read_excel('inflation_data_in_v2.xlsx', index_col=0)


In [57]:
## Here, Create a list of all dates available in the inflation input file:


date_list = list(pd.to_datetime(df.index.values))

df_Train = df.iloc[0:i,:]
df_Test = df.iloc[i+1,:]


,Core
Date,
2016-06-30,1.58475
2016-09-30,1.72221
2016-12-30,1.85337
2017-03-31,1.64406
2017-06-30,1.62126
2017-09-29,1.47651
2017-12-29,1.63842
2018-03-30,1.95941
2018-06-29,1.95136


In [34]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]


In [160]:
inf_fcst_loop_list_qtr = []


## Quarterly:
for i in range(189,201,1):
    df_Train = df.iloc[0:i,]
#     df_Test = df.iloc[i:,]
    df_Test = date_list[i]
    

    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.sarimax.SARIMAX(df,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)

                results = mod.fit(disp=False)

    #             print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
            except:
                continue
                
    mod = sm.tsa.statespace.sarimax.SARIMAX(df_Train,
                                order=(1, 1, 1),
                                seasonal_order=(0, 0, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results = mod.fit() 
    
    pred = results.get_prediction(start = df_Test , end = df_Test , dynamic=False)
    pred_ci = pred.conf_int()
#     pred_ci
    pred_ci['Midpoint'] = pred_ci.mean(axis=1)
    inf_fcst_loop_list_qtr.append(pred_ci['Midpoint'].iloc[0])
    test.append(df_Test)
         

In [93]:
inf_fcst_loop_list_semi = []

## Semi:
for i in range(190,199,1):
    df_Train = df.iloc[0:i,]
#     df_Test = df.iloc[i:,]
    df_Test_st = date_list[i+1]
    df_Test_end = date_list[i+2]
    

    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.sarimax.SARIMAX(df,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)

                results = mod.fit(disp=False)

    #             print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
            except:
                continue
                
    mod = sm.tsa.statespace.sarimax.SARIMAX(df_Train,
                                order=(1, 1, 1),
                                seasonal_order=(0, 0, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results = mod.fit() 
    
    pred = results.get_prediction(start = df_Test_st , end = df_Test_end , dynamic=False)
    pred_ci = pred.conf_int()
#     pred_ci
    pred_ci['Midpoint'] = pred_ci.mean(axis=1)
    inf_fcst_loop_list_semi.append(pred_ci['Midpoint'].iloc[0])

    
inf_fcst_loop_list_semi_1 = inf_fcst_loop_list_semi[0::2]

In [128]:
inf_fcst_loop_list_annual = []


## Annual:
for i in range(192,197,1):
    df_Train = df.iloc[0:i,]
#     df_Test = df.iloc[i:,]
    df_Test_st = date_list[i]
    df_Test_end = date_list[i+4]
    

    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.sarimax.SARIMAX(df,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)

                results = mod.fit(disp=False)

    #             print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
            except:
                continue
                
    mod = sm.tsa.statespace.sarimax.SARIMAX(df_Train,
                                order=(1, 1, 1),
                                seasonal_order=(0, 0, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results = mod.fit() 
    
    pred = results.get_prediction(start = df_Test_st , end = df_Test_end , dynamic=False)
    pred_ci = pred.conf_int()
#     pred_ci
    pred_ci['Midpoint'] = pred_ci.mean(axis=1)
    inf_fcst_loop_list_annual.append(pred_ci['Midpoint'].iloc[0])
    test.append(df_Test_end)
    
inf_fcst_loop_list_annual_1 = inf_fcst_loop_list_annual[0::4]


## Create all relevant Dataframes:

In [165]:
dates = list(UER.Quarter[-12:])

## Quarterly:
gdp_pce_fcst_qtr = pd.DataFrame({'Quarter' : dates,
                                 'GDP_fcst_qtr': gdp_fcst_loop_list_qtr , 
                                 'PCE_fcst_qtr': inf_fcst_loop_list_qtr})


# display(gdp_pce_fcst_qtr,
#         gdp_pce_fcst_semi, 
#         gdp_pce_fcst_annual)

,Quarter,GDP_fcst_qtr,PCE_fcst_qtr
0,2016_1,2.107690,1.234343
1,2016_2,2.608709,1.554293
2,2016_3,2.452613,1.644466
3,2016_4,2.926525,1.787516
4,2017_1,3.654303,1.916445
5,2017_2,4.107014,1.583232
6,2017_3,4.049717,1.597866
7,2017_4,4.143414,1.415084
8,2018_1,5.307285,1.683767
9,2018_2,3.759553,2.088672


,Quarter,GDP_fcst_semi,PCE_fcst_semi
0,2016_3,2.452613,1.602746
1,2017_1,3.654303,1.826238
2,2017_3,4.049717,1.545833
3,2018_1,5.307285,1.378261
4,2018_3,3.933972,2.166876


,Quarter,GDP_fcst_annual,PCE_fcst_annual
0,2017_4,4.143414,1.787516
1,2018_4,4.106581,1.415084


In [166]:
## Check which Quadrant they fall under for quarterly:

def f(row):
    if (row['GDP_fcst_delta'] > 0 and row['PCE_fcst_delta'] > 0):
        val = 1
    elif (row['GDP_fcst_delta'] > 0 and row['PCE_fcst_delta'] <= 0):
        val = 2
    elif (row['GDP_fcst_delta'] <= 0 and row['PCE_fcst_delta'] <= 0):
        val = 3
    else:
        val = 4
    return val

gdp_pce_fcst_qtr['GDP_fcst_delta'] =  gdp_pce_fcst_qtr['GDP_fcst_qtr'] - gdp_pce_fcst_qtr['GDP_fcst_qtr'].shift(1) 
gdp_pce_fcst_qtr['PCE_fcst_delta'] =   gdp_pce_fcst_qtr['PCE_fcst_qtr'] - gdp_pce_fcst_qtr['PCE_fcst_qtr'].shift(1)

gdp_pce_fcst_qtr['Quadrant'] = gdp_pce_fcst_qtr.apply(f, axis=1)
gdp_pce_fcst_qtr

,Quarter,GDP_fcst_qtr,PCE_fcst_qtr,GDP_fcst_delta,PCE_fcst_delta,Quadrant
0,2016_1,2.107690,1.234343,NaN,NaN,4
1,2016_2,2.608709,1.554293,0.501019,0.319951,1
2,2016_3,2.452613,1.644466,-0.156096,0.090172,4
3,2016_4,2.926525,1.787516,0.473912,0.143051,1
4,2017_1,3.654303,1.916445,0.727778,0.128929,1
5,2017_2,4.107014,1.583232,0.452710,-0.333214,2
6,2017_3,4.049717,1.597866,-0.057296,0.014634,4
7,2017_4,4.143414,1.415084,0.093697,-0.182782,2
8,2018_1,5.307285,1.683767,1.163870,0.268683,1
9,2018_2,3.759553,2.088672,-1.547731,0.404905,4


In [167]:
#### Write the output to Excel
writer = pd.ExcelWriter('OUTPUT_GDP_PCE_fcst_30042019_v1.xlsx')
gdp_pce_fcst_qtr.to_excel(writer,'Qtrly Fcst')
gdp_pce_fcst_semi.to_excel(writer,'Semi Fcst')
gdp_pce_fcst_annual.to_excel(writer,'Annual Fcst')
writer.save() 
